# Day3_0 별첨: 다중 상속 (Multiple Inheritance)

## 📚 학습 목표

1. **다중 상속 문법** 이해하고 사용하기
2. **MRO (Method Resolution Order)** 완벽 이해하기
3. **다이아몬드 문제** 해결 방법 익히기
4. **Mixin 패턴** 실무 활용하기
5. **다중 상속의 장단점** 파악하고 올바르게 사용하기

---

## 🎯 왜 다중 상속을 배우나요?

| 개념 | 단일 상속 | 다중 상속 |
|------|----------|----------|
| 상속 가능 부모 수 | 1개 | 여러 개 |
| 기능 조합 | 제한적 | 자유로움 |
| 복잡도 | 낮음 | 높음 (MRO 이해 필요) |
| 실무 활용 | 일반적 | Mixin 패턴 |

**분석가 관점**: Django의 View 클래스, SQLAlchemy의 Mixin 등 많은 프레임워크에서 다중 상속을 활용합니다!

---

## 1. 다중 상속 기본

### 문법

```python
class Child(Parent1, Parent2, Parent3):
    pass
```

자식 클래스는 **모든 부모 클래스의 속성과 메서드를 상속**받습니다.

In [ ]:
# 기본 예시: 능력(Ability) 조합하기
class Flyable:
    """날 수 있는 능력"""
    def fly(self):
        return f"{self.name}가 날아갑니다! 🦅"

class Swimmable:
    """수영할 수 있는 능력"""
    def swim(self):
        return f"{self.name}가 수영합니다! 🏊"

class Walkable:
    """걸을 수 있는 능력"""
    def walk(self):
        return f"{self.name}가 걷습니다! 🚶"

# 다중 상속: 여러 능력을 동시에 가진 생물
class Duck(Flyable, Swimmable, Walkable):
    """오리 - 날고, 수영하고, 걷기도 함"""
    def __init__(self, name):
        self.name = name
    
    def quack(self):
        return f"{self.name}: 꽥꽥! 🦆"

# 사용
donald = Duck("도널드")
print(donald.quack())  # 자체 메서드
print(donald.fly())    # Flyable로부터
print(donald.swim())   # Swimmable로부터
print(donald.walk())   # Walkable로부터

# 어떤 클래스들을 상속받았는지 확인
print(f"\n상속 계층: {Duck.__bases__}")

In [ ]:
# 다른 조합: 펭귄 - 수영하고 걷지만, 날지는 못함
class Penguin(Swimmable, Walkable):
    """펭귄 - 수영하고 걷지만 날지 못함"""
    def __init__(self, name):
        self.name = name
    
    def slide(self):
        return f"{self.name}가 배로 미끄러집니다! 🐧"

pingu = Penguin("핑구")
print(pingu.swim())
print(pingu.walk())
print(pingu.slide())

# 날 수 있는지 확인
print(f"\n날 수 있나요? {hasattr(pingu, 'fly')}")

---

## 2. MRO (Method Resolution Order)

**MRO**는 메서드를 찾는 **순서**를 정의합니다.  
여러 부모 클래스에 같은 이름의 메서드가 있을 때, **어느 것을 먼저 호출할지** 결정합니다.

### MRO 규칙 (C3 Linearization)

1. **자식 클래스 우선**
2. **부모 클래스는 왼쪽에서 오른쪽 순서**
3. **부모의 부모는 모든 직계 부모 이후**

In [ ]:
# MRO 확인하기
class A:
    def method(self):
        return "A의 메서드"

class B(A):
    def method(self):
        return "B의 메서드"

class C(A):
    def method(self):
        return "C의 메서드"

class D(B, C):  # B를 먼저, C를 나중에 상속
    pass

# MRO 확인
print("=== D의 MRO (Method Resolution Order) ===")
print(D.__mro__)
print()

# 더 읽기 쉽게
for i, cls in enumerate(D.__mro__):
    print(f"{i+1}. {cls.__name__}")

# 메서드 호출 - MRO 순서대로 탐색
d = D()
print(f"\nd.method() 호출 결과: {d.method()}")
print("→ D에 없으므로 → B (첫 번째 부모) → B의 method() 호출!")

In [ ]:
# 순서를 바꾸면?
class E(C, B):  # C를 먼저, B를 나중에
    pass

print("=== E의 MRO ===")
for i, cls in enumerate(E.__mro__):
    print(f"{i+1}. {cls.__name__}")

e = E()
print(f"\ne.method() 호출 결과: {e.method()}")
print("→ E에 없으므로 → C (첫 번째 부모) → C의 method() 호출!")

### 💡 MRO 시각화

```python
class D(B, C):
```

**MRO**: D → B → C → A → object

```
       object
         |
         A
        / \
       B   C
        \ /
         D
```

메서드 탐색: **D → B → C → A → object** 순서로 찾습니다.

---

## 3. 다이아몬드 문제 (Diamond Problem)

다중 상속의 가장 유명한 문제입니다.

```
       A
      / \
     B   C
      \ /
       D
```

- B와 C가 모두 A를 상속
- D가 B와 C를 모두 상속
- **문제**: A의 `__init__`이 두 번 호출될까요?

In [ ]:
# 다이아몬드 문제 시연
class GrandParent:
    def __init__(self):
        print("GrandParent.__init__ 호출")
        self.value = "조부모"

class Parent1(GrandParent):
    def __init__(self):
        print("Parent1.__init__ 호출")
        super().__init__()
        self.value1 = "부모1"

class Parent2(GrandParent):
    def __init__(self):
        print("Parent2.__init__ 호출")
        super().__init__()
        self.value2 = "부모2"

class Child(Parent1, Parent2):
    def __init__(self):
        print("Child.__init__ 호출")
        super().__init__()
        self.value_child = "자식"

# 생성
print("=== Child 인스턴스 생성 ===")
child = Child()

# MRO 확인
print("\n=== MRO ===")
for i, cls in enumerate(Child.__mro__):
    print(f"{i+1}. {cls.__name__}")

print("\n✅ GrandParent.__init__은 단 한 번만 호출됩니다!")
print("→ super()가 MRO를 따라 올바르게 호출")

### ⚠️ super() 없이 직접 호출하면?

**잘못된 방법**: `super()` 대신 클래스 이름으로 직접 호출

In [ ]:
# ❌ 잘못된 방법: super() 대신 직접 호출
class GrandParentBad:
    def __init__(self):
        print("GrandParentBad.__init__ 호출")

class Parent1Bad(GrandParentBad):
    def __init__(self):
        print("Parent1Bad.__init__ 호출")
        GrandParentBad.__init__(self)  # ❌ 직접 호출

class Parent2Bad(GrandParentBad):
    def __init__(self):
        print("Parent2Bad.__init__ 호출")
        GrandParentBad.__init__(self)  # ❌ 직접 호출

class ChildBad(Parent1Bad, Parent2Bad):
    def __init__(self):
        print("ChildBad.__init__ 호출")
        Parent1Bad.__init__(self)  # ❌ 직접 호출
        Parent2Bad.__init__(self)  # ❌ 직접 호출

print("=== 잘못된 방법 ===")
child_bad = ChildBad()
print("\n❌ GrandParentBad.__init__이 두 번 호출되었습니다!")
print("→ 비효율적이고 버그 가능성 높음")

### 📌 정리: 다이아몬드 문제 해결

| 방법 | 결과 | 권장 여부 |
|------|------|----------|
| `super()` 사용 | 조부모 `__init__` **1번** 호출 | ✅ 권장 |
| 클래스명 직접 호출 | 조부모 `__init__` **2번** 호출 | ❌ 비권장 |

**핵심**: 항상 `super()`를 사용하세요! MRO를 따라 올바르게 호출됩니다.

---

## 4. Mixin 패턴 (실무 핵심!)

**Mixin**은 다중 상속의 가장 실용적인 활용 패턴입니다.

### Mixin이란?

- **단일 기능**을 제공하는 작은 클래스
- **독립적으로 사용되지 않음** (다른 클래스와 혼합해서 사용)
- **관례**: 클래스명에 `Mixin` 접미사 추가

### Mixin의 특징

- `__init__` 없음 (또는 최소한)
- 하나의 명확한 기능만 제공
- 다른 클래스에 "믹스인"하여 기능 추가

In [ ]:
# Mixin 패턴 예시
import json
from datetime import datetime

class JSONSerializableMixin:
    """JSON 직렬화 기능 제공"""
    def to_json(self):
        """객체를 JSON 문자열로 변환"""
        data = {}
        for key, value in self.__dict__.items():
            if isinstance(value, datetime):
                data[key] = value.isoformat()
            else:
                data[key] = value
        return json.dumps(data, ensure_ascii=False, indent=2)

class TimestampMixin:
    """생성/수정 시각 자동 관리"""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.created_at = datetime.now()
        self.updated_at = datetime.now()
    
    def touch(self):
        """수정 시각 업데이트"""
        self.updated_at = datetime.now()

class ReprMixin:
    """자동 __repr__ 생성"""
    def __repr__(self):
        attrs = ', '.join(f"{k}={v!r}" for k, v in self.__dict__.items())
        return f"{self.__class__.__name__}({attrs})"

# 기본 클래스
class Article:
    """블로그 글"""
    def __init__(self, title, content, author):
        self.title = title
        self.content = content
        self.author = author

# Mixin을 조합한 강화된 클래스
class EnhancedArticle(JSONSerializableMixin, TimestampMixin, ReprMixin, Article):
    """JSON 직렬화 + 타임스탬프 + 자동 repr 기능을 가진 블로그 글"""
    pass

# 사용
article = EnhancedArticle(
    title="Python 다중 상속",
    content="다중 상속은 강력하지만 신중하게 사용해야 합니다.",
    author="단테"
)

print(article)  # ReprMixin
print("\n=== JSON 직렬화 ===")
print(article.to_json())  # JSONSerializableMixin

# 시간 흐름 시뮬레이션
import time
time.sleep(1)
article.touch()  # TimestampMixin

print(f"\n생성 시각: {article.created_at}")
print(f"수정 시각: {article.updated_at}")

### 💡 실무 예시: Django의 Mixin

Django는 Mixin 패턴을 광범위하게 사용합니다:

```python
# Django의 실제 코드 패턴
from django.views.generic import ListView
from django.contrib.auth.mixins import LoginRequiredMixin
from django.contrib.messages.views import SuccessMessageMixin

class ArticleListView(LoginRequiredMixin, SuccessMessageMixin, ListView):
    # LoginRequiredMixin: 로그인 체크
    # SuccessMessageMixin: 성공 메시지 표시
    # ListView: 리스트 뷰 기능
    model = Article
```

### 💡 실무 예시: SQLAlchemy Mixin

In [ ]:
# SQLAlchemy 스타일 Mixin
class SoftDeleteMixin:
    """Soft Delete 기능 (실제 삭제 대신 is_deleted 플래그)"""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.is_deleted = False
    
    def delete(self):
        """Soft Delete"""
        self.is_deleted = True
        return f"{self.__class__.__name__} soft deleted"
    
    def restore(self):
        """복구"""
        self.is_deleted = False
        return f"{self.__class__.__name__} restored"

class ActiveRecordMixin:
    """Active Record 패턴"""
    def save(self):
        return f"{self.__class__.__name__} saved to database"
    
    @classmethod
    def find_by_id(cls, id):
        return f"Finding {cls.__name__} with id={id}"

# 모델 클래스
class User(SoftDeleteMixin, ActiveRecordMixin):
    """사용자 모델 (Soft Delete + Active Record 기능)"""
    def __init__(self, username, email):
        super().__init__()
        self.username = username
        self.email = email

# 사용
user = User("dante", "dante@example.com")
print(user.save())  # ActiveRecordMixin
print(user.delete())  # SoftDeleteMixin
print(f"삭제됨? {user.is_deleted}")
print(user.restore())  # SoftDeleteMixin
print(f"삭제됨? {user.is_deleted}")
print(User.find_by_id(123))  # ActiveRecordMixin

---

## 5. 다중 상속 vs 단일 상속 vs 조합

### 언제 무엇을 사용할까?

| 패턴 | 사용 시기 | 장점 | 단점 |
|------|----------|------|------|
| **단일 상속** | 명확한 is-a 관계 | 단순, 이해하기 쉬움 | 기능 조합 제한적 |
| **다중 상속 (Mixin)** | 여러 독립 기능 조합 | 코드 재사용 극대화 | MRO 이해 필요 |
| **조합 (Composition)** | has-a 관계 | 유연, 결합도 낮음 | 코드 다소 길어짐 |

### 실무 가이드라인

1. **기본**: 단일 상속 사용
2. **기능 추가**: Mixin 패턴 사용 (다중 상속)
3. **복잡한 관계**: 조합(Composition) 사용
4. **다이아몬드 상속**: 피하거나 super() 철저히 사용

In [ ]:
# 조합(Composition) vs 다중 상속 비교

# 1. 다중 상속 방식
class LoggerMixin:
    def log(self, message):
        print(f"[LOG] {message}")

class CacheMixin:
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._cache = {}
    
    def get_cache(self, key):
        return self._cache.get(key)
    
    def set_cache(self, key, value):
        self._cache[key] = value

class DataService(LoggerMixin, CacheMixin):
    """다중 상속 방식"""
    def fetch_data(self, key):
        # 캐시 확인
        cached = self.get_cache(key)
        if cached:
            self.log(f"Cache hit: {key}")
            return cached
        
        # 데이터 가져오기
        data = f"Data for {key}"
        self.set_cache(key, data)
        self.log(f"Cache miss: {key}")
        return data

# 2. 조합(Composition) 방식
class Logger:
    def log(self, message):
        print(f"[LOG] {message}")

class Cache:
    def __init__(self):
        self._cache = {}
    
    def get(self, key):
        return self._cache.get(key)
    
    def set(self, key, value):
        self._cache[key] = value

class DataServiceComposition:
    """조합(Composition) 방식"""
    def __init__(self):
        self.logger = Logger()  # has-a 관계
        self.cache = Cache()    # has-a 관계
    
    def fetch_data(self, key):
        # 캐시 확인
        cached = self.cache.get(key)
        if cached:
            self.logger.log(f"Cache hit: {key}")
            return cached
        
        # 데이터 가져오기
        data = f"Data for {key}"
        self.cache.set(key, data)
        self.logger.log(f"Cache miss: {key}")
        return data

# 비교
print("=== 다중 상속 방식 ===")
service1 = DataService()
print(service1.fetch_data("user_1"))
print(service1.fetch_data("user_1"))

print("\n=== 조합 방식 ===")
service2 = DataServiceComposition()
print(service2.fetch_data("user_2"))
print(service2.fetch_data("user_2"))

---

## 🎯 실습 퀴즈

---

### Q1. 기본 다중 상속 ⭐

**문제**: `Robot` 클래스를 만드세요.

**요구사항**:
- `Movable` 클래스: `move()` 메서드 ("이동합니다" 반환)
- `Speakable` 클래스: `speak()` 메서드 ("말합니다" 반환)
- `Robot` 클래스: 위 두 클래스를 상속하고, `name` 속성 추가
- 모든 메서드는 `self.name`을 포함한 메시지 반환

In [ ]:
# 여기에 코드를 작성하세요

# 테스트
robot = Robot("R2D2")
print(robot.move())
print(robot.speak())

### Q2. MRO 이해하기 ⭐⭐

**문제**: 다음 클래스 계층에서 MRO를 예측하고 확인하세요.

**요구사항**:
```python
class Base:
    def method(self):
        return "Base"

class Left(Base):
    def method(self):
        return "Left"

class Right(Base):
    def method(self):
        return "Right"

class Child(Left, Right):
    pass
```

1. MRO를 출력하세요
2. `Child().method()` 결과를 예측하고 확인하세요

In [ ]:
# 여기에 코드를 작성하세요

# 테스트
child = Child()
print(f"MRO: {[cls.__name__ for cls in Child.__mro__]}")
print(f"child.method() = {child.method()}")

### Q3. super()로 다이아몬드 해결 ⭐⭐⭐

**문제**: 다이아몬드 상속 구조에서 `super()`를 올바르게 사용하세요.

**요구사항**:
```python
class Vehicle:
    def __init__(self, brand):
        self.brand = brand

class Car(Vehicle):
    def __init__(self, brand, model):
        # super() 사용
        self.model = model

class Electric:
    def __init__(self, battery_capacity):
        self.battery_capacity = battery_capacity

class ElectricCar(Car, Electric):
    def __init__(self, brand, model, battery_capacity):
        # super() 사용하여 모든 __init__ 호출
```

- `ElectricCar("Tesla", "Model 3", 75)`가 모든 속성을 올바르게 설정해야 함

In [ ]:
# 여기에 코드를 작성하세요

# 테스트
tesla = ElectricCar("Tesla", "Model 3", 75)
print(f"브랜드: {tesla.brand}")
print(f"모델: {tesla.model}")
print(f"배터리: {tesla.battery_capacity}kWh")
print(f"MRO: {[cls.__name__ for cls in ElectricCar.__mro__]}")

### Q4. Mixin 패턴 구현 ⭐⭐⭐⭐

**문제**: 로깅 기능을 제공하는 Mixin을 만드세요.

**요구사항**:

**LoggingMixin**:
- `log(level, message)` 메서드: `"[레벨] 메시지"` 형식으로 출력
- `info(message)`, `warning(message)`, `error(message)` 메서드 제공

**User 클래스**:
- `name`, `email` 속성
- `login()` 메서드: 로그인 시 `info()` 로그 출력
- `logout()` 메서드: 로그아웃 시 `info()` 로그 출력

**AdminUser 클래스**:
- `User`와 `LoggingMixin` 상속
- `delete_user(username)` 메서드: 삭제 시 `warning()` 로그 출력

In [ ]:
# 여기에 코드를 작성하세요

# 테스트
admin = AdminUser("admin", "admin@example.com")
admin.login()
admin.delete_user("old_user")
admin.logout()

### Q5. 종합 실무 예제 ⭐⭐⭐⭐⭐

**문제**: 데이터베이스 모델을 위한 Mixin 시스템을 만드세요.

**요구사항**:

**TimestampMixin**:
- `created_at`, `updated_at` 자동 관리
- `touch()` 메서드로 `updated_at` 업데이트

**ValidationMixin**:
- `validate()` 메서드 (추상 메서드, 서브클래스에서 구현)
- `is_valid()` 메서드: `validate()` 호출 후 True/False 반환

**SerializableMixin**:
- `to_dict()` 메서드: 모든 속성을 딕셔너리로 변환
- datetime 객체는 ISO 형식 문자열로 변환

**Product 클래스**:
- 위 3개 Mixin 모두 상속
- 속성: `name`, `price`, `stock`
- `validate()` 구현: price > 0, stock >= 0 검증
- `sell(quantity)` 메서드: 판매 후 `touch()` 호출

In [ ]:
# 여기에 코드를 작성하세요
from datetime import datetime
import time

# TimestampMixin, ValidationMixin, SerializableMixin, Product 작성

# 테스트
product = Product("노트북", 1500000, 10)

print(f"유효성 검사: {product.is_valid()}")
print(f"\n초기 데이터:")
print(product.to_dict())

time.sleep(1)
result = product.sell(3)
print(f"\n{result}")

print(f"\n판매 후 데이터:")
print(product.to_dict())

# 잘못된 제품
bad_product = Product("에러상품", -1000, -5)
print(f"\n잘못된 제품 유효성: {bad_product.is_valid()}")

---

## 📊 학습 정리

### 핵심 개념 복습

| 개념 | 설명 | 핵심 |
|------|------|------|
| 다중 상속 | 여러 부모 클래스 상속 | `class C(A, B):` |
| MRO | 메서드 탐색 순서 | 왼쪽 → 오른쪽, 깊이 우선 |
| 다이아몬드 문제 | 공통 조상 중복 호출 | `super()` 사용으로 해결 |
| Mixin | 기능 제공 클래스 | 단일 기능, `__init__` 최소화 |
| super() | 부모 메서드 호출 | MRO 순서 따름 |

### 다중 상속 사용 가이드

**✅ 좋은 사용 예**:
- Mixin 패턴으로 독립적 기능 추가
- 로깅, 직렬화, 타임스탬프 등 횡단 관심사
- Django View, SQLAlchemy Model처럼 프레임워크 패턴

**❌ 피해야 할 경우**:
- 복잡한 다이아몬드 상속
- `__init__`이 복잡한 여러 클래스 상속
- MRO 이해 없이 무분별한 사용

### Mixin 작성 원칙

1. **단일 책임**: 하나의 명확한 기능만 제공
2. **독립성**: 다른 클래스에 의존하지 않음
3. **명명 규칙**: 클래스명에 `Mixin` 접미사
4. **`__init__` 최소화**: super().__init__() 패턴 사용
5. **문서화**: 어떤 기능을 제공하는지 명확히 기술

### super() 사용 규칙

```python
# ✅ 올바른 패턴
class Child(Parent1, Parent2):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)  # MRO 따름
        # 자식 초기화

# ❌ 잘못된 패턴
class Child(Parent1, Parent2):
    def __init__(self):
        Parent1.__init__(self)  # 직접 호출 (다이아몬드 문제)
        Parent2.__init__(self)
```

---

## 🎓 다음 단계

다중 상속을 마스터했다면:
- **메타클래스**: 클래스 생성 과정 제어
- **디스크립터**: 속성 접근 제어
- **데코레이터 패턴**: 다중 상속 대안
- **조합 vs 상속**: 디자인 패턴 비교

**수고하셨습니다!** 🎉